# import library and data

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import datetime
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from scipy import interpolate

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import plotly.graph_objects as go
from matplotlib import dates
from plotly.subplots import make_subplots
import plotly
import plotly.express as px
pd.options.display.float_format = '{:.5f}'.format

import FinanceDataReader as fdr

from sklearn.preprocessing import MinMaxScaler

# os.getcwd()
### 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
font_list
plt.rc("font", family="DejaVu Sans")
parameters = {'axes.labelsize': 20,
          'axes.titlesize': 25,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
        'legend.fontsize': 20,
          }
plt.rcParams.update(parameters)
plt.rcParams['font.family'] = 'DejaVu Sans'

current_dir = os.getcwd()
plt.style.use("seaborn-dark")
plt.rcParams['font.family'] = 'DejaVu Sans'

In [2]:
# root_path = '../../../'
# master_path = os.path.join(root_path, '01. Data','00. master_data')
# current_dir = os.getcwd()

# file_path = os.path.join(root_path, '01. Data','02. 재무지표2', '01. preprocessed')
# result_path = os.path.join(current_dir, 'result')



# root_path = "C:/Users/10188/Desktop/기업가치"
# master_path = os.path.join(root_path, "master_data")
# rawfile_path = os.path.join(master_path, "result_xlsx")
# file_path = os.path.join(root_path, "clean_data","clean_total","ITS")

# out_path = os.path.join(root_path, "clean_data","clean_total")


root_path = '../../../'
master_path = os.path.join(root_path, '01. Data','00. master_data')
rawfile_path = os.path.join(master_path, "result_xlsx")
file_path = os.path.join(root_path, '01. Data','02. 재무지표2', '01. preprocessed')

result_path = os.path.join(current_dir, 'result')

In [3]:
file_lst = ['Accenture_non_shift_clean_total_v1.1.csv','Infosys_non_shift_clean_total_v1.1.csv','TCS_non_shift_clean_total_v1.1.csv']

In [4]:
desc = pd.read_excel(os.path.join(master_path, "desc_v2.2.xlsx"), sheet_name='desc', header=0, usecols=['no','raw','variable','group','importance'])
desc.head(2)

,raw,no,variable,group,importance
0,Date_x,1,Date_x,DT,9
1,BS_Earnings Quality Score,2,BS_Earnings Quality Score,BS,5


In [5]:
# desc 데이터 기준, group이 한글인 경우(재무팀에서 뽑아준 항목들) + 전체(영어 포함)를 target variable로 가져감
# target_variable = desc.loc[~desc['group'].apply(lambda x: x.encode().isalpha()),"variable"].values.tolist()

dic_group = dict()
for _, xcol in enumerate(desc['variable'].values):
    dic_group[xcol] = desc.loc[_,'group']


dic_imp = dict()
for _, xcol in enumerate(desc['variable'].values):
    dic_imp[xcol] = desc.loc[_,'importance']



dic_name = dict()
for _, xcol in enumerate(desc['raw'].values):
    dic_name[xcol] = desc.loc[_, 'variable']

dic_name['Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.1'] = "FS_Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.1"
dic_name['Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.2'] = "FS_Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.2"
dic_name['Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.3'] = "FS_Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.3"

dic_name['IS_Other, Net.1'] = "IS_Other, Net.1"
dic_name['IS_Current Tax - Total.1'] = "IS_Current Tax - Total.1"
dic_name['IS_Deferred Tax - Total.1'] = "IS_Deferred Tax - Total.1"


# # C1, C2, C3, C4, C5
# C = ['매출액증가율(YoY)(연도)', 'Gross Investment (GI)','무형자산회전율s','매출총이익(천원)','무형자산'] 
# dic_c = {C[0]:"C1", C[1]: "C2", C[2]: "C3", C[3]:"C4", C[4]: "C5"}
# C_ = ['C1','C2','C3','C4','C5']

# Y1, Y2, Y3, Y4
Y = ['close','close_weighted','PBR','EV_EBITDA']

remove_col = ['인당매출','무형자산회전율','인당영업이익','인당순이익','투자비 대비 매출 (Rev/GI)']
ev_col = ['FS_Enterprise Value', 'EBITDA2(천원)', 'EBIT(천원)' ]

# Accenture

In [6]:
raw_file_name = "Accenture_non_shift.xlsx"
company = raw_file_name.split("_")[0]

df = pd.read_excel(os.path.join(rawfile_path,raw_file_name))
df['Date_x'] = pd.to_datetime(df['Date_x']).dt.to_period('Q')
df['Date_x'] = pd.PeriodIndex(df['Date_x'], freq='Q').to_timestamp()

for col in df.columns:
    if col in list(dic_name.keys()) :
        df.rename(columns={col:dic_name[col]}, inplace=True)
df = df[df.columns.difference(['Date_x'])]     

In [7]:
filter_df = pd.read_csv(os.path.join(file_path,file_lst[0]))
company = file_lst[0].split("_")[0]
company

'Accenture'

## na값과 mode 비율 계산

In [8]:
out = df.isna().sum().to_frame().reset_index()
out.columns = ['cols','na_num']
out['group'] = out['cols'].map(dic_group)
out['importance'] = out['cols'].map(dic_imp)
out = out[['cols','group','importance','na_num']]

In [9]:
# Counter(new_df[i])
# data.most_common(1)[0][1]
for _,col in enumerate(out['cols']):
    tmp = Counter(df[col])
    out.loc[_,'mode_num'] = tmp.most_common(1)[0][1]/len(df)
out

,cols,group,importance,na_num,mode_num
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01250
1,BPS증가율(YoY)(비율),성장성,2.00000,4,0.01250
2,BS_Accounts Payable,BS,5.00000,0,0.02500
3,"BS_Accounts Receivable - Trade, Gross",BS,5.00000,0,0.51250
4,"BS_Accounts Receivable - Trade, Net",BS,5.00000,0,0.01250
...,...,...,...,...,...
377,채무총계,안정성,1.00000,0,0.11250
378,총자산회전율(비율),활동성,0.00000,0,0.06250
379,투자비 대비 매출 (Rev/GI),NaN,NaN,0,0.01250
380,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01250


## 클러스터 관련 데이터 불러오기

In [10]:
cl = pd.read_csv(os.path.join(result_path,company+'_final_cols.csv'))
cl

,cols,cluster,distance
0,매출액(천원),46,0.07849
1,매출총이익(천원),46,0.08736
2,"BS_Total Operating Leases, Supplemental",44,0.19938
3,BS_Total Common Shares Outstanding,114,1.66330
4,무형자산,45,0.12790
...,...,...,...
129,인당순이익s,23,0.72246
130,인당영업이익s,117,1.73456
131,재고자산회전율(비율),15,0.67025
132,투자비 대비 매출 (Rev/GI)s,68,0.94715


## cor 계산

In [11]:
### Y과 X들 간 corr 값 계산
target = 'EV_EBITDA'

Y = filter_df[[target]]
X = filter_df[filter_df.columns.difference(['Date_x'])]

cor = X[list(set(cl.cols.values.tolist())|set([target]))].corr(method='pearson')[[target]].sort_values(by=target, ascending=False, key=abs)
cor = cor.loc[cor.index[cor.index != target],:].reset_index()
cor.columns = ['cols','corr']
cor

,cols,corr
0,매출액(천원),0.92151
1,Gross Investment (GI),0.91834
2,매출총이익(천원),0.91264
3,BS_Additional Paid-In Capital,0.90161
4,무형자산감상비 누계,-0.88885
...,...,...
129,BS_Current Port. of LT Debt/Capital Leases,-0.01577
130,CF_Sale of Fixed Assets,-0.01437
131,EBITDA2증가율(YoY)(비율),-0.01187
132,매입채무회전율,-0.01106


## 합치기

In [12]:
final = pd.merge(out, cl, how='outer', right_on='cols', left_on='cols')
final = pd.merge(final, cor, how='outer', right_on='cols', left_on='cols')
final.sort_values('corr', ascending=False, key=abs).reset_index(drop=True, inplace=True)
final

,cols,group,importance,na_num,mode_num,cluster,distance,corr
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01250,85.00000,1.13653,-0.13653
1,BPS증가율(YoY)(비율),성장성,2.00000,4,0.01250,87.00000,0.77723,0.22277
2,BS_Accounts Payable,BS,5.00000,0,0.02500,NaN,NaN,NaN
3,"BS_Accounts Receivable - Trade, Gross",BS,5.00000,0,0.51250,NaN,NaN,NaN
4,"BS_Accounts Receivable - Trade, Net",BS,5.00000,0,0.01250,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
377,채무총계,안정성,1.00000,0,0.11250,62.00000,0.65789,0.34211
378,총자산회전율(비율),활동성,0.00000,0,0.06250,120.00000,1.80810,-0.80810
379,투자비 대비 매출 (Rev/GI),NaN,NaN,0,0.01250,NaN,NaN,NaN
380,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01250,68.00000,0.94715,0.05285


In [13]:
Y = ['close','close_weighted','PBR']

remove_col = ['인당매출','무형자산회전율','인당영업이익','인당순이익','투자비 대비 매출 (Rev/GI)']
ev_col = ['FS_Enterprise Value', 'EBITDA2(천원)', 'EBIT(천원)' ]

In [14]:
final = final[final['cols'].apply(lambda x:( x not in Y) and (x not in remove_col) and (x not in ev_col))].reset_index(drop=True)
final.sort_values('corr', ascending=False, key=abs, inplace=True)
final.reset_index(drop=True, inplace=True)
final

,cols,group,importance,na_num,mode_num,cluster,distance,corr
0,매출액(천원),성장성,0.00000,0,0.01250,46.00000,0.07849,0.92151
1,Gross Investment (GI),성장성,2.00000,0,0.01250,46.00000,0.08166,0.91834
2,매출총이익(천원),수익성,0.00000,0,0.02500,46.00000,0.08736,0.91264
3,BS_Additional Paid-In Capital,BS,5.00000,0,0.12500,47.00000,0.09839,0.90161
4,무형자산감상비 누계,안정성,1.00000,0,0.60000,121.00000,1.88885,-0.88885
...,...,...,...,...,...,...,...,...
366,유동자산(천원),안정성,2.00000,0,0.02500,NaN,NaN,NaN
367,인력수,DL,9.00000,2,0.10000,NaN,NaN,NaN
368,자기자본비율,안정성,2.00000,0,0.01250,NaN,NaN,NaN
369,자본금(천원),안정성,2.00000,0,1.00000,NaN,NaN,NaN


In [15]:
final.to_excel(os.path.join(result_path,"backup",company+"_backup.xlsx"), index=False)

# Infosys

In [16]:
raw_file_name = "Infosys_non_shift.xlsx"
company = raw_file_name.split("_")[0]

df = pd.read_excel(os.path.join(rawfile_path,raw_file_name))
df['Date_x'] = pd.to_datetime(df['Date_x']).dt.to_period('Q')
df['Date_x'] = pd.PeriodIndex(df['Date_x'], freq='Q').to_timestamp()

for col in df.columns:
    if col in list(dic_name.keys()) :
        df.rename(columns={col:dic_name[col]}, inplace=True)
df = df[df.columns.difference(['Date_x'])]     

In [17]:
filter_df = pd.read_csv(os.path.join(file_path,file_lst[1]))
company = file_lst[1].split("_")[0]
company

'Infosys'

## na값과 mode 비율 계산

In [18]:
out = df.isna().sum().to_frame().reset_index()
out.columns = ['cols','na_num']
out['group'] = out['cols'].map(dic_group)
out['importance'] = out['cols'].map(dic_imp)
out = out[['cols','group','importance','na_num']]

In [19]:
# Counter(new_df[i])
# data.most_common(1)[0][1]
for _,col in enumerate(out['cols']):
    tmp = Counter(df[col])
    out.loc[_,'mode_num'] = tmp.most_common(1)[0][1]/len(df)
out

,cols,group,importance,na_num,mode_num
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01250
1,BPS증가율(YoY)(비율),성장성,2.00000,4,0.01250
2,BS_Accounts Payable,BS,5.00000,0,0.18750
3,"BS_Accounts Receivable - Trade, Gross",BS,5.00000,0,0.31250
4,"BS_Accounts Receivable - Trade, Net",BS,5.00000,0,0.02500
...,...,...,...,...,...
409,채무총계,안정성,1.00000,0,0.85000
410,총자산회전율(비율),활동성,0.00000,4,0.06250
411,투자비 대비 매출 (Rev/GI),NaN,NaN,0,0.01250
412,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01250


## 클러스터 관련 데이터 불러오기

In [20]:
cl = pd.read_csv(os.path.join(result_path,company+'_final_cols.csv'))
cl

,cols,cluster,distance
0,매출액(천원),115,1.28298
1,매출총이익(천원),115,1.33956
2,Gross Investment (GI),120,1.22479
3,자본금(천원),120,1.11723
4,유형자산감상비 누계,63,0.80255
...,...,...,...
130,인당매출s,132,0.66618
131,자기자본비율,16,1.19623
132,재고자산회전율(비율),131,0.33769
133,투자비 대비 매출 (Rev/GI)s,18,1.33156


## cor 계산

In [21]:
### Y과 X들 간 corr 값 계산
target = 'EV_EBITDA'

Y = filter_df[[target]]
X = filter_df[filter_df.columns.difference(['Date_x'])]

cor = X[list(set(cl.cols.values.tolist())|set([target]))].corr(method='pearson')[[target]].sort_values(by=target, ascending=False, key=abs)
cor = cor.loc[cor.index[cor.index != target],:].reset_index()
cor.columns = ['cols','corr']
cor

,cols,corr
0,매출액증가율(YoY)(연도),0.71562
1,"BPS(지배, Adj.)(원/주)",0.71313
2,재고자산회전율(비율),0.66231
3,BS_Construction in Progress - Gross,-0.66142
4,FS_Dividend Yield - Common Stock - Net - Issue...,-0.65435
...,...,...
129,이자보상율,-0.01398
130,CF_Foreign Exchange Effects,0.00890
131,"CF_Other Investing Cash Flow Items, Total",-0.00838
132,CF_Accounts Receivable,-0.00838


## 합치기

In [22]:
final = pd.merge(out, cl, how='outer', right_on='cols', left_on='cols')
final = pd.merge(final, cor, how='outer', right_on='cols', left_on='cols')
final.sort_values('corr', ascending=False, key=abs).reset_index(drop=True, inplace=True)
final

,cols,group,importance,na_num,mode_num,cluster,distance,corr
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01250,55.00000,0.28687,0.71313
1,BPS증가율(YoY)(비율),성장성,2.00000,4,0.01250,133.00000,0.65883,0.34117
2,BS_Accounts Payable,BS,5.00000,0,0.18750,NaN,NaN,NaN
3,"BS_Accounts Receivable - Trade, Gross",BS,5.00000,0,0.31250,112.00000,1.27816,-0.27816
4,"BS_Accounts Receivable - Trade, Net",BS,5.00000,0,0.02500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
409,채무총계,안정성,1.00000,0,0.85000,NaN,NaN,NaN
410,총자산회전율(비율),활동성,0.00000,4,0.06250,32.00000,1.02151,-0.02151
411,투자비 대비 매출 (Rev/GI),NaN,NaN,0,0.01250,NaN,NaN,NaN
412,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01250,18.00000,1.33156,-0.33156


In [23]:
Y = ['close','close_weighted','PBR']

remove_col = ['인당매출','무형자산회전율','인당영업이익','인당순이익','투자비 대비 매출 (Rev/GI)']
ev_col = ['FS_Enterprise Value', 'EBITDA2(천원)', 'EBIT(천원)' ]

In [24]:
final = final[final['cols'].apply(lambda x:( x not in Y) and (x not in remove_col) and (x not in ev_col))].reset_index(drop=True)
final.sort_values('corr', ascending=False, key=abs, inplace=True)
final.reset_index(drop=True, inplace=True)
final

,cols,group,importance,na_num,mode_num,cluster,distance,corr
0,매출액증가율(YoY)(연도),성장성,0.00000,4,0.01250,56.00000,0.28438,0.71562
1,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01250,55.00000,0.28687,0.71313
2,재고자산회전율(비율),활동성,2.00000,0,0.01250,131.00000,0.33769,0.66231
3,BS_Construction in Progress - Gross,BS,5.00000,0,0.03750,80.00000,1.66142,-0.66142
4,FS_Dividend Yield - Common Stock - Net - Issue...,NaN,NaN,3,0.02500,93.00000,1.65435,-0.65435
...,...,...,...,...,...,...,...,...
398,이익잉여금(천원),수익성,2.00000,0,0.01250,NaN,NaN,NaN
399,인당순이익s,수익성,2.00000,3,0.01250,NaN,NaN,NaN
400,인력수,DL,9.00000,0,0.05000,NaN,NaN,NaN
401,자산총계(천원),안정성,2.00000,0,0.01250,NaN,NaN,NaN


In [25]:
final.to_excel(os.path.join(result_path,"backup",company+"_backup.xlsx"), index=False)

# TCS

In [26]:
raw_file_name = "TCS_non_shift.xlsx"
company = raw_file_name.split("_")[0]
print(company)

df = pd.read_excel(os.path.join(rawfile_path,raw_file_name))
df['Date_x'] = pd.to_datetime(df['Date_x']).dt.to_period('Q')
df['Date_x'] = pd.PeriodIndex(df['Date_x'], freq='Q').to_timestamp()

for col in df.columns:
    if col in list(dic_name.keys()) :
        df.rename(columns={col:dic_name[col]}, inplace=True)
df = df[df.columns.difference(['Date_x'])]     

TCS


In [27]:
filter_df = pd.read_csv(os.path.join(file_path,file_lst[2]))
company = file_lst[2].split("_")[0]
company

'TCS'

## na값과 mode 비율 계산

In [28]:
out = df.isna().sum().to_frame().reset_index()
out.columns = ['cols','na_num']
out['group'] = out['cols'].map(dic_group)
out['importance'] = out['cols'].map(dic_imp)
out = out[['cols','group','importance','na_num']]

In [29]:
# Counter(new_df[i])
# data.most_common(1)[0][1]
for _,col in enumerate(out['cols']):
    tmp = Counter(df[col])
    out.loc[_,'mode_num'] = tmp.most_common(1)[0][1]/len(df)
out

,cols,group,importance,na_num,mode_num
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01471
1,BPS증가율(YoY)(비율),성장성,2.00000,4,0.01471
2,BS_Accounts Payable,BS,5.00000,0,0.04412
3,"BS_Accounts Receivable - Trade, Gross",BS,5.00000,0,0.05882
4,"BS_Accounts Receivable - Trade, Net",BS,5.00000,0,0.02941
...,...,...,...,...,...
389,채무총계,안정성,1.00000,0,0.05882
390,총자산회전율(비율),활동성,0.00000,9,0.08824
391,투자비 대비 매출 (Rev/GI),NaN,NaN,0,0.01471
392,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01471


## 클러스터 관련 데이터 불러오기

In [30]:
cl = pd.read_csv(os.path.join(result_path,company+'_final_cols.csv'))
cl

,cols,cluster,distance
0,매출액(천원),131,0.53728
1,매출총이익(천원),131,0.53595
2,Opex Margin,131,0.58334
3,부채총계(천원),129,0.43566
4,채무총계,106,0.47764
...,...,...,...
152,재고자산회전율(비율),60,1.11090
153,투자비 대비 매출 (Rev/GI)s,121,0.38925
154,현금배당성향(%),101,0.84022
155,EV_EBITDA,92,0.00000


## cor 계산

In [31]:
### Y과 X들 간 corr 값 계산
target = 'EV_EBITDA'

Y = filter_df[[target]]
X = filter_df[filter_df.columns.difference(['Date_x'])]

cor = X[list(set(cl.cols.values.tolist())|set([target]))].corr(method='pearson')[[target]].sort_values(by=target, ascending=False, key=abs)
cor = cor.loc[cor.index[cor.index != target],:].reset_index()
cor.columns = ['cols','corr']
cor

,cols,corr
0,"BPS(지배, Adj.)(원/주)",0.79911
1,"FS_Free Cash Flow Yield - %, TTM",-0.73477
2,FS_Dividend Yield - Common Stock - Net - Issue...,-0.63767
3,ROGI(%),0.63102
4,투자비 대비 매출 (Rev/GI)s,0.61075
...,...,...
151,"BS_Other Equity, Total",0.02186
152,FS_Cash & Cash Equivalents,0.02091
153,IS_Total Pension Expense,0.01765
154,CF_Purchase of Investments,-0.01761


## 합치기

In [32]:
final = pd.merge(out, cl, how='outer', right_on='cols', left_on='cols')
final = pd.merge(final, cor, how='outer', right_on='cols', left_on='cols')
final.sort_values('corr', ascending=False, key=abs).reset_index(drop=True, inplace=True)
final

,cols,group,importance,na_num,mode_num,cluster,distance,corr
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01471,91.00000,0.20089,0.79911
1,BPS증가율(YoY)(비율),성장성,2.00000,4,0.01471,22.00000,0.58533,0.41467
2,BS_Accounts Payable,BS,5.00000,0,0.04412,137.00000,0.64470,0.35530
3,"BS_Accounts Receivable - Trade, Gross",BS,5.00000,0,0.05882,NaN,NaN,NaN
4,"BS_Accounts Receivable - Trade, Net",BS,5.00000,0,0.02941,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
389,채무총계,안정성,1.00000,0,0.05882,106.00000,0.47764,0.52236
390,총자산회전율(비율),활동성,0.00000,9,0.08824,32.00000,0.82212,0.17788
391,투자비 대비 매출 (Rev/GI),NaN,NaN,0,0.01471,NaN,NaN,NaN
392,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01471,121.00000,0.38925,0.61075


In [33]:
Y = ['close','close_weighted','PBR']

remove_col = ['인당매출','무형자산회전율','인당영업이익','인당순이익','투자비 대비 매출 (Rev/GI)']
ev_col = ['FS_Enterprise Value', 'EBITDA2(천원)', 'EBIT(천원)' ]

In [34]:
final = final[final['cols'].apply(lambda x:( x not in Y) and (x not in remove_col) and (x not in ev_col))].reset_index(drop=True)
final.sort_values('corr', ascending=False, key=abs, inplace=True)
final.reset_index(drop=True, inplace=True)
final

,cols,group,importance,na_num,mode_num,cluster,distance,corr
0,"BPS(지배, Adj.)(원/주)",안정성,2.00000,0,0.01471,91.00000,0.20089,0.79911
1,"FS_Free Cash Flow Yield - %, TTM",FS,5.00000,4,0.02941,84.00000,1.73477,-0.73477
2,FS_Dividend Yield - Common Stock - Net - Issue...,FS,5.00000,1,0.04412,83.00000,1.63767,-0.63767
3,ROGI(%),수익성,0.00000,5,0.01471,120.00000,0.36898,0.63102
4,투자비 대비 매출 (Rev/GI)s,성장성,0.00000,3,0.01471,121.00000,0.38925,0.61075
...,...,...,...,...,...,...,...,...
378,영업이익률(비율),수익성,2.00000,1,0.02941,NaN,NaN,NaN
379,유동자산(천원),안정성,2.00000,0,0.01471,NaN,NaN,NaN
380,이익잉여금(천원),수익성,2.00000,0,0.02941,NaN,NaN,NaN
381,인력수,DL,9.00000,0,0.05882,NaN,NaN,NaN


In [35]:
final.to_excel(os.path.join(result_path,"backup",company+"_backup.xlsx"), index=False)